## Попередня обробка даних

### Вступ

Попередня обробка даних - це важливий етап в аналізі даних. Необхідно заздалегідь підготувати дані таким чином, щоб найкраще розкрити структуру проблеми алгоритмам машинного навчання, які будуть до цих даних застосовуватись. Це передбачає ряд заходів, таких як:
* Присвоєння числових значень категоричним даним;
* Обробка відсутніх значень;
* Нормалізація функцій (щоб функції на малих масштабах не домінували при підключенні моделі до даних).



У розділі 2 "Дослідницький аналіз даних" виконане дослідження, щоб допомогти зрозуміти розподіл даних, а також те, як атрибути співвідносяться між собою. Були визначені функції інтересу. У цьому розділі використовуються функції для оптимізації даних великих розмірів.

### Мета:
Знайти найбільш прогнозовані функції даних і відфільтрувати їх так, щоб це підвищило прогнозовану силу аналітичної моделі.

#### Завантаження даних та основних бібліотек

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Завантаження бібліотек для обробки даних
import pandas as pd #обробка даних, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from scipy.stats import norm

# візуалізація
import seaborn as sns 
plt.style.use('fivethirtyeight')
sns.set_style("white")


plt.rcParams['figure.figsize'] = (8,4) 
#plt.rcParams['axes.titlesize'] = 'large'

data = pd.read_csv('data/clean-data.csv', index_col=False)
data.drop('Unnamed: 0',axis=1, inplace=True)
#data.head()

#### Кодування міток
Занесення 30 ознак до NumPy масиву X та перетворення класових міток з їх оригінального рядкового (string) представлення (M та B) в цілочисельне (integer).

In [ ]:
#Призначення предикторів до змінної типу ndarray (матриця)
array = data.values
X = array[:,1:31]
y = array[:,0]

In [ ]:
#перетворення класових міток з їх оригінального рядкового (string) представлення (M та B) в цілочисельне (integer)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

#метод перетворення LabelEncorder двох фіктивних змінних
#le.transform (['M', 'B'])

Після кодування міток класу (діагнозу) у масиві злоякісні пухлини тепер представлені як клас 1 (тобто наявність ракових клітин), а доброякісні пухлини представлені як клас 0 (тобто відсутність ракових клітин) відповідно.


#### Оцінка точності моделі: розбиття даних на навчальні та тестові набори

Найпростіший метод оцінювання продуктивності алгоритму машинного навчання - використання різних наборів даних для навчання та тестування. Кроки:
* Розділити наявні дані на навчальний і тестовий набір. (70% навчання, 30% тесту)
* Навчання алгоритму на першій частині даних
* Зробити прогноз щодо другої частини
* Оцінити прогнози щодо очікуваних результатів

Розмір розбиття може залежати від розміру та специфіки набору даних, хоча зазвичай 67% даних використовується для тренувань, а решта 33% - для тестування.

In [ ]:
from sklearn.model_selection import train_test_split

##Розбиття набору даних: 70% для навчання, 30% для тестування
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25, random_state=7)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

#### Стандартизація ознак

* Стандартизація - це корисна методика перетворення атрибутів з гауссовим розподілом та різними відхиленнями у стандартний розподіл Гаусса із середнім значенням 0 та стандартним відхиленням 1.

* Як вже відомо з розділу 2 "Дослідницький аналіз даних" вихідні дані мають різні розподіли, що може впливати на більшість алгоритмів машинного навчання. Алгоритми машинного навчання та оптимізації поводяться набагато краще, якщо функції знаходяться в одному масштабі.

Оцінимо ті самі алгоритми зі стандартизованою копією набору даних. Використовується sklearn для масштабування та перетворення даних таким чином, що кожен атрибут має середнє значення 0 та стандартне відхилення 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

# нормалізація даних (центр близький до 0 і масштаб для видалення дисперсії).
scaler =StandardScaler()
Xs = scaler.fit_transform(X)

#### Декомпозиція ознак використовуючи аналіз основних компонентів - Principal Component Analysis (PCA)
В розділі 2 "Дослідницький аналіз даних" були наведені пари ознак, які добре розділяють дані. Тому є сенс використовувати один із методів зменшення розмірності, щоб спробувати використовувати якомога більше функцій та підтримувати якомога більше інформації при роботі з лише двома вимірами. Далі використовується PCA.

In [ ]:
from sklearn.decomposition import PCA
# вилучення ознак
pca = PCA(n_components=10)
fit = pca.fit(Xs)

# підсумкові дані
#print("Explained Variance: %s") % fit.explained_variance_ratio_
#print(fit.components_)

In [ ]:
X_pca = pca.transform(Xs)

PCA_df = pd.DataFrame()

PCA_df['PCA_1'] = X_pca[:,0]
PCA_df['PCA_2'] = X_pca[:,1]

plt.plot(PCA_df['PCA_1'][data.diagnosis == 'M'],PCA_df['PCA_2'][data.diagnosis == 'M'],'o', alpha = 0.7, color = 'r')
plt.plot(PCA_df['PCA_1'][data.diagnosis == 'B'],PCA_df['PCA_2'][data.diagnosis == 'B'],'o', alpha = 0.7, color = 'b')

plt.xlabel('PCA_1')
plt.ylabel('PCA_2')
plt.legend(['Malignant','Benign'])
plt.show()

Після застосування лінійної трансформації PCA отримано підпростір (від 3D до 2D), де зразки "найбільш розповсюджені" уздовж нових осей функції.

In [ ]:
#кількість дисперсій, яку пояснює кожен головний компонент
var= pca.explained_variance_ratio_
#сукупна варіативність
#var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
#print(var1)

### Вирішення, скільки основних компонентів потрібно зберегти
Для того, щоб визначити, скільки основних компонентів слід зберегти, прийнято узагальнити результати аналізу основних компонентів, склавши графік огляду.

In [ ]:
#кількість дисперсій, яку пояснює кожен головний компонент
var= pca.explained_variance_ratio_
#сукупна варіативність
#var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
#print(var1)

plt.plot(var)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')

leg = plt.legend(['Eigenvalues from PCA'], loc='best', borderpad=0.3,shadow=False,markerscale=0.4)
leg.get_frame().set_alpha(0.4)
leg.draggable(state=True)
plt.show()

Найбільш очевидна зміна нахилу відбувається в компоненті 2, який є «ліктем» графіку власних значень. Таким чином, виходячи з графіку, можна стверджувати, що перші три компоненти слід зберегти.

### Підсумок:

1. Занесення 30 ознак до NumPy масиву X та перетворення класових міток з їх оригінального рядкового представлення (M та B) в цілочисельне.
2. Розбиття даних на навчальні та тестові набори.
3. Стандартизація даних.
4. Отримання власних векторів та значень з матриці коваріації чи матриці кореляції.
5. Сортування власних значень у порядку зменшення та вибір власних векторів kk, які відповідають найбільшим власним значенням kk, де k - розмірність нового підпростору ознак (k≤dk≤d).
6. Побудова проекційної матриці W з вибраних власних векторів k.
7. Перетворення вихідного набору даних X через W, щоб отримати k-мірний підпростір ознак Y.